# Group line numbers from GTFS by quays (CHB)
Goal is to show lines per stop (quaycode), to avoid using an API for this in the [Haltebuddy](https://haltebuddy.focustest.nl/landingpage) prototype.

In [1]:
import pandas as pd
import zipfile

In [2]:
# CHB downloaded and converted from http://data.ndovloket.nl/haltes/
# You can use https://next.observablehq.com/@jurb/haltebestand-ndov-loket-xml-export-naar-csv to generate and download an up to date version
quays = pd.read_csv('chb/quays.csv')

In [3]:
# GTFS downloaded from https://transitfeeds.com/p/ov/814/latest
# Omitted from this repo, get the current version of the zip and put it in this directory
zf = zipfile.ZipFile('gtfs.zip') 

In [4]:
routes = pd.read_csv(zf.open('routes.txt'))
stop_times = pd.read_csv(zf.open('stop_times.txt'))
stops = pd.read_csv(zf.open('stops.txt'))
trips = pd.read_csv(zf.open('trips.txt'))

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color,route_url
0,75937,ARR,2,Brunssum/Hoensbroek via Heerlen - Bleijerheide,NaN,3,NaN,NaN,NaN
1,74020,RET,574,STOPenGO Lansingerland,NaN,3,NaN,NaN,NaN
2,74018,RET,526,STOPenGO Maassluis,NaN,3,NaN,NaN,NaN
3,73358,BRENG,78,Papendal - Beekdal Lyceum,NaN,3,NaN,NaN,http://www.breng.nl/dienstregeling/lijn?ID=A078
4,67306,BRENG,352,Wageningen Station - Arnhem CS,NaN,3,NaN,NaN,http://www.breng.nl/dienstregeling/lijn?ID=A352
...,...,...,...,...,...,...,...,...,...
2425,62989,ARR,6381,"Nijetrijne - Wolvega, Station",NaN,3,NaN,NaN,NaN
2426,62990,ARR,6371,"Nijelamer - Wolvega, Station",NaN,3,NaN,NaN,NaN
2427,62991,ARR,7133,Jislum - Oentsjerk,NaN,3,NaN,NaN,NaN
2428,62992,ARR,7152,"Burdaard - Dokkum, Sionsberg",NaN,3,NaN,NaN,NaN


In [5]:
# De meeste quays van de GVB beginnen met 'NL:Q:300'. De stop_code in de GTFS bestanden voor deze quays is het getal dat na 'NL:Q:300' komt. Voor de quaycodes die niet met deze string beginnen, is het getal na 'NL:Q:' de stop_code.

def construct_stop_code(quaycode):
    if quaycode[0:8] == 'NL:Q:300':
        return quaycode[8:13]
    else:
        return quaycode[5:13]

quays['stop_code'] = quays['quaycode'].apply(lambda x: construct_stop_code(x))

In [6]:
# merge quays met de gtfs stops informatie (quays en stops betekent hetzelfde, maar quays komt hier can CHB en stops van de GTFS set. Ik weet niet waarom we met meer quays overblijven na de left merge op quays :)
quays_stop_codes = quays.query('quaystatus == "available"').merge(stops, how="left", on="stop_code")
print(quays.query('quaystatus == "available"').shape)
print(quays_stop_codes.shape)

(1478, 34)
(1488, 44)


In [7]:
# Maak een selectie van het grote stop_times bestand door alleen te selecteren op wat we in de quay set hebben.
stop_times_filtered_on_quays = stop_times[stop_times.stop_id.isin(quays_stop_codes.stop_id.dropna())]

In [8]:
# 273 stop_id's zijn NaN, die moeten er uit. Om te mergen moeten we ook nog een type forcen
print(quays_stop_codes.stop_id.isnull().sum())
quays_stop_codes = quays_stop_codes.dropna(subset=['stop_id'])
quays_stop_codes.stop_id = quays_stop_codes.stop_id.astype(int)
print(quays_stop_codes.stop_id.isnull().sum())

273
0


In [9]:
stop_times_with_quays = stop_times_filtered_on_quays.merge(quays_stop_codes, on="stop_id")

In [10]:
alles = stop_times_with_quays.merge(trips, on="trip_id").merge(routes, on="route_id")

In [11]:
# Al met al hebben we voor 1205 van de 1478 quays matchende lijnen gevonden. Dit kan best logisch te verklaren zijn, nog even uitzoeken.

result = alles.groupby(['quaycode']).apply(lambda x: list(dict.fromkeys(x['route_short_name'].to_list()))).reset_index(name = "route_short_name")
result

,quaycode,route_short_name
0,NL:Q:30000001,"[40, 41, 22]"
1,NL:Q:30000005,"[3, 245, 37]"
2,NL:Q:30000006,"[40, 41, 22]"
3,NL:Q:30000007,"[3, 37]"
4,NL:Q:30000008,"[3, 37, 41, 1]"
...,...,...
1200,NL:Q:57320060,[382]
1201,NL:Q:57320070,[382]
1202,NL:Q:57352120,[120]
1203,NL:Q:57352130,[120]


In [12]:
# result.to_json('quay_route_names.json', orient="records")

In [13]:
result.set_index('quaycode').to_json('quay_route_names.json')